In [22]:
import pandas as pd
import numpy as np
import timeit
import geopy.distance
import requests
import json
#pd.set_option('display.max_columns', None)

In [8]:
df = pd.read_csv('Data/renewables.csv', index_col=0)
df

,Longitude,Latitude,Solar Energy Potential,Wind Power Density,Port Code,Port Long.,Port Lat.,Gen-Port Direct Dist.,Gen-Port Driving Dist.
0,-159.0,56.0,1063.974986,472.33026,4944,-152.415736,57.783555,447.504518,491.8272
1,-159.0,59.0,1079.305006,369.35553,4944,-152.415736,57.783555,408.020243,NaN
2,-135.0,57.0,917.975038,958.34094,3146,-130.000127,55.902870,331.533877,1852.1911
3,-128.0,55.0,999.004996,1433.11040,3679,-128.671822,53.977895,121.818806,207.8273
4,-126.0,55.0,1163.984960,77.40454,3679,-128.671822,53.977895,207.160715,422.9948
...,...,...,...,...,...,...,...,...,...
5965,174.0,-36.0,1325.315015,343.84467,2198,174.539025,-35.865232,50.888321,92.2879
5966,175.0,-39.0,1227.860030,146.18901,2488,174.652134,-38.174366,96.532935,199.0102
5967,175.0,-38.0,1220.925010,331.73700,2488,174.652134,-38.174366,36.137437,54.9704
5968,175.0,-37.0,1302.685002,310.05020,6825,174.767247,-36.840815,27.243295,34.9165


In [9]:
df_port_index = pd.read_csv('Data/port_index.csv', index_col=0)
#del df_port_index['Unnamed: 0.1']
#del df_port_index['Unnamed: 0.1.1']
#del df_port_index['Unnamed: 0.1.1.1']
#del df_port_index['Unnamed: 0.1.1.1.1']
#del df_port_index['47914']
#del df_port_index['47914.1']
#del df_port_index['47914.1.1']
#del df_port_index['47914.1.1.1']
#df_port_index.to_csv('Data/port_index.csv')
df_port_index

,Port Code,2832,2625,2100,4201,47914,2210,18641,2894,3264,...,2680,2001,7132,3227,43448,43364,22289,5226,2815,3864
0,4944,15696.189539,13574.376207,9028.136360,16757.084547,18440.355818,12715.280817,18827.136365,18902.514336,19632.174773,...,11275.017938,8678.667343,20462.990750,18055.996578,15880.992417,7208.273828,19935.123506,19632.174773,18502.262544,16706.676724
1,4944,15696.189539,13574.376207,9028.136360,16757.084547,18440.355818,12715.280817,18827.136365,18902.514336,19632.174773,...,11275.017938,8678.667343,20462.990750,18055.996578,15880.992417,7208.273828,19935.123506,19632.174773,18502.262544,16706.676724
2,3146,15696.189539,13574.376207,9028.136360,16757.084547,17392.648452,14504.701610,17779.429000,17854.806970,18584.467408,...,13064.438732,10468.088136,19415.283384,19845.417371,17670.413211,8997.694621,18887.416140,18584.467408,17454.555179,15658.969359
3,3679,14981.975145,13435.420770,9275.676537,16042.870153,17209.980991,14406.132168,17596.761538,17672.139509,18401.799946,...,12965.869289,10369.518694,19232.615923,19746.847928,17571.843768,8899.125179,18704.748679,18401.799946,17271.887718,15476.301898
4,3679,14981.975145,13435.420770,9275.676537,16042.870153,17209.980991,14406.132168,17596.761538,17672.139509,18401.799946,...,12965.869289,10369.518694,19232.615923,19746.847928,17571.843768,8899.125179,18704.748679,18401.799946,17271.887718,15476.301898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5965,2198,5732.531650,7891.307073,15679.191090,5600.660317,21313.387326,7607.288371,21700.167873,20551.535844,22505.206281,...,9183.062047,9661.873085,23336.022258,14995.426618,13691.919759,10034.110437,22808.155014,22505.206281,21375.294052,18741.626377
5966,2488,5732.531650,7891.307073,15679.191090,5600.660317,21727.816873,7309.206000,22114.597421,20027.790996,22919.635828,...,9304.141381,9933.834373,23750.451805,14471.681770,13168.174911,10356.698316,23222.584561,22919.635828,21789.723600,18552.373136
5967,2488,5732.531650,7891.307073,15679.191090,5600.660317,21727.816873,7309.206000,22114.597421,20027.790996,22919.635828,...,9304.141381,9933.834373,23750.451805,14471.681770,13168.174911,10356.698316,23222.584561,22919.635828,21789.723600,18552.373136
5968,6825,5732.531650,7891.307073,15679.191090,5600.660317,21396.889346,7762.956146,21783.669894,20707.203619,22588.708302,...,9338.729821,9817.540860,23419.524278,15151.094393,13847.587534,10189.778212,22891.657034,22588.708302,21458.796073,18751.939165


In [10]:
df = pd.read_csv('Results/final_df.csv', index_col=0)
df.sample(20)

,Longitude,Latitude,Solar Energy Potential,Wind Power Density,Port Code,Port Long.,Port Lat.,Gen-Port Direct Dist.,Gen-Port Driving Dist.,Solar Array Size,...,LOHC Cost,H2 Liq Cost,H2 Gas Cost,Transport Cost per kg H2,Cheapest Medium,Direct Dist.,Driving Dist.,Yearly Transport Cost,Total Yearly Cost,Total Cost per kg H2
615,-92.0,16.0,1759.665036,644.664400,4590,-92.402600,14.702600,149.927003,233.6871,1.767636e+06,...,2.462435,4.773010,NaN,2.462435,LOHC,7961.887976,NaN,1.231217e+08,4.082441e+08,8.164881
2782,25.0,-31.0,1936.690085,214.359920,3067,25.685590,-33.790970,316.148780,395.9364,1.606064e+06,...,2.683534,5.235710,NaN,2.683534,LOHC,6792.647554,NaN,1.341767e+08,4.508979e+08,9.017958
4781,83.0,37.0,1399.044980,122.165115,3735,88.307857,22.528535,1683.476774,3592.1895,2.223265e+06,...,7.181842,14.918090,NaN,5.828212,NH3,9939.716533,NaN,2.914106e+08,6.550855e+08,13.101710
4526,75.0,27.0,1666.225021,90.867080,2109,72.535833,21.636895,644.483782,791.0532,1.866763e+06,...,3.253369,6.487327,NaN,3.253369,LOHC,9519.326847,NaN,1.626684e+08,4.992224e+08,9.984449
4345,68.0,52.0,1234.065015,283.073360,7131,73.674919,68.505472,1863.680289,2824.3137,2.520489e+06,...,6.085804,12.530949,NaN,5.199572,NH3,8423.752487,NaN,2.599786e+08,6.290233e+08,12.580466
3245,33.0,53.0,1120.185002,196.030800,2941,32.006651,46.905730,681.573145,923.7258,2.776727e+06,...,3.419091,6.805774,NaN,3.419091,LOHC,6117.130940,NaN,1.709546e+08,5.767342e+08,11.534683
1193,-60.0,2.0,1488.834999,51.611782,5922,-57.522313,6.285200,547.893285,925.1126,2.089182e+06,...,3.410503,6.768083,NaN,3.410503,LOHC,4882.931739,NaN,1.705252e+08,5.239997e+08,10.479994
5900,148.0,-22.0,1745.794997,125.442800,1967,149.247641,-21.113455,162.282970,204.8901,1.781680e+06,...,2.450024,4.797536,NaN,2.450024,LOHC,18288.639684,NaN,1.225012e+08,4.525825e+08,9.051649
461,-99.0,40.0,1621.694942,402.675230,4035,-92.024965,46.726035,936.231094,1221.6957,1.918023e+06,...,3.843305,7.727730,NaN,3.843305,LOHC,8265.619175,NaN,1.921653e+08,5.167697e+08,10.335394
5639,123.0,10.0,1247.569994,261.974670,3930,122.777362,10.492710,59.707622,152.6670,2.493205e+06,...,2.373274,4.626407,NaN,2.373274,LOHC,14877.200317,NaN,1.186637e+08,4.997585e+08,9.995170


In [11]:
df.iloc[3771]

Longitude                                11.0
Latitude                                 54.0
Solar Energy Potential            1026.380013
Wind Power Density                   563.2857
Port Code                                4825
Port Long.                          10.858373
Port Lat.                            53.94723
Gen-Port Direct Dist.               10.993716
Gen-Port Driving Dist.                17.6664
Solar Array Size               3030503.050632
Solar CapEx                 2121352135.442735
Wind Turbine Power                   1.769614
No. of Turbines                    292.949704
Wind CapEx                   738233252.850987
Yearly Cost Solar            230545960.019202
Yearly Cost Wind             100074269.519125
Elec Cost Solar                     74.119862
Elec Cost Wind                       32.17359
Cheaper source                           Wind
Yearly gen. cost             294613972.293949
Gen. cost per kg H2                  5.892279
Shipping Dist.                    

In [26]:
def get_driving_distance(start_point, end_point):
    """Gets the driving distance (km) from the start point to the end point (input in [lat, long])."""

    # call the OSMR API - needs [long, lat]
    r = requests.get(
        f"http://router.project-osrm.org/route/v1/car/{end_point[1]},{end_point[0]};{start_point[1]},{start_point[0]}?overview=false""")
    # then you load the response using the json libray
    # by default you get only one alternative so you access 0-th element of the `routes`
    routes = json.loads(r.content)
    route_1 = routes.get("routes")[0]
    driving_distance = route_1["distance"] / 1000

    return driving_distance

end_tuple = (25.28893697992723, 51.565533460963046)

# start timer
start = timeit.default_timer()

for i in range(len(df)):
    direct_distance_total = geopy.distance.distance((df.at[i, 'Latitude'], df.at[i, 'Longitude']),
                                                    end_tuple).km  # Needs [lat, long]
        # Check if it is possible to truck the whole way to end destination. If so, get costs.
    if direct_distance_total < 700:
        try:
            driving_distance_total = get_driving_distance((df.at[i, 'Latitude'], df.at[i, 'Longitude']),
                                                          end_tuple)  # Needs [lat, long]
        except:
            driving_distance_total = np.nan
    else:
        driving_distance_total = np.nan
        
        
# stop timer
stop = timeit.default_timer()
print('Total Time: ', stop - start)

Total Time:  28.764829200000122
